In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import documents as docs
import textcomparisons as tc
import comparisonmachine_multiprocessing as machine
import random
import time

start = time.time()

data_folder = "data"
results_folder = "results"
para_sep = "###"

ac = tc.ArticleComparisons(thresh_jaccard = .5, thresh_same_sent = .9, thresh_same_doc = .25)

article_df = pd.read_pickle(os.path.join(data_folder, "article_df_20180715"))

display(article_df.head(2))
print("Setup time: %d seconds" % np.round(time.time() - start))

,id,publisher,scrape_time,on_homepage,url,title,text,fingerprint,event,doc,paywall
0,0,wsbtv.com,2018-07-16T01:02:26-04:00,True,https://www.wsbtv.com/news/local/cobb-county/p...,"Police: Missing teen with autism found, reunit...","Police: Missing teen with autism found, reunit...","{'wordCounts': {'1533': 10, '367': 3, '2619': ...",NaN,None,0.0
1,1,wn.com,2018-07-16T04:51:37-04:00,False,https://wn.com/Apostolic_Constitutions,Apostolic s,Apostolic Constitutions ### back to playlist #...,"{'wordCounts': {'147633': 5, '5029': 6, '546':...",NaN,None,0.0


Setup time: 1 seconds


In [2]:
n = 100
start = time.time()
#sample = random.sample(list(article_df["id"]), n)
sample = [i for i in range(n)]#f article_df.loc[i, "event"] == 1]
print(len(sample)) #, sample)
# article_dict_spacy = ac.dict_by_ids(article_df, sample, para_sep = para_sep, parser = "spacy")
article_dict = ac.dict_by_ids(article_df, sample, para_sep = para_sep, parser = "nltk")
elapsed = round(time.time() - start, 2)
print("Time taken to run cell: %f s (%f s per document)" % (elapsed, elapsed/n))


100
Time taken to run cell: 1.260000 s (0.012600 s per document)


In [6]:
comparer = machine.ParallelComparisons()
mat = comparer.run(article_dict)

In [28]:
id = 2
row = article_df.loc[2, ["id", "text", "doc"]]
row["text"].split(" ")

['No',
 'refuge',
 'from',
 'politics',
 'but',
 'France',
 'victory',
 'a',
 'fitting',
 'climax',
 '###',
 'Alexei',
 'Nikolsky',
 '###',
 'No',
 'refuge',
 'from',
 'politics',
 'but',
 'France',
 'victory',
 'a',
 'fitting',
 'climax',
 '###',
 'French',
 'President',
 'Emmanuel',
 'Macron',
 'reacts',
 'during',
 'the',
 'final',
 'match',
 'between',
 'France',
 'and',
 'Croatia',
 'at',
 'the',
 '2018',
 'soccer',
 'World',
 'Cup',
 'in',
 'the',
 'Luzhniki',
 'Stadium',
 'in',
 'Moscow,',
 'Russia,',
 'Sunday,',
 'July',
 '15,',
 '2018.',
 '(Alexei',
 'Nikolsky,',
 'Sputnik,',
 'Kremlin',
 'Pool',
 'Photo',
 'via',
 'AP)',
 '###',
 'Petr',
 'David',
 'Josek',
 '###',
 'No',
 'refuge',
 'from',
 'politics',
 'but',
 'France',
 'victory',
 'a',
 'fitting',
 'climax',
 '###',
 'In',
 'this',
 'photo',
 'taken',
 'with',
 'slow',
 'shutter',
 'speed',
 "France's",
 'Kylian',
 'Mbappe',
 'runs',
 'with',
 'the',
 'ball',
 'during',
 'the',
 'semifinal',
 'match',
 'between',
 'Franc

In [12]:
ids = [2, 3, 4, 5]
retdocs = [None, None, None, None]

article_df.loc[ids, "doc"] = retdocs

In [23]:
article_df.loc[article_df["id"] == ids]

ValueError: Lengths must match to compare

In [87]:
def fxn():
    cands = np.where((article_df["id"] % 100 == 0) & (article_df["id"] % 3 == 0))[0]
    
%timeit fxn()

3.2 ms ± 77.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [84]:
def fxn():
    for i in article_df["id"]:
        i % 100 == 0 and i % 3 == 0
        
%timeit fxn()

17.9 ms ± 959 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [88]:
"hi" + "2018901"

'hi2018901'